In [95]:
import json, os

import pandas as pd
import numpy as np

from konlpy.tag import Okt
import nltk

from sklearn.model_selection import train_test_split

from keras.models import Sequential, load_model
from keras.layers import Dense 
from keras import optimizers, losses, metrics

In [53]:
def read_csv():
    # pd로 csv 읽기
    return pd.concat([pd.read_csv("data/neg_opinion12.csv"), pd.read_csv("data/pos_opinion12.csv"),\
                      pd.read_csv("data/neg_opinion19.csv"), pd.read_csv("data/pos_opinion19.csv")])\
            .reset_index().drop('index', axis=1) # 인덱스 초기화

In [63]:
def check_pos():
    # 이미 형태소 전처리 된 json 파일 있는지 확인
    fname = "pos_tag.json"
    if os.path.isfile(fname):
        print("// 형태소 전처리 파일 존재")
        with open(fname, encoding="utf-8-sig") as f:
            data = pd.DataFrame(json.load(f), columns=["comment", "risk"])
    else: 
        print("// 형태소 전처리 파일 미존재")
        # okt로 형태소 분석하기
        data = read_csv()
        okt = Okt()
        def tokenize(doc):
            return ['/'.join(t) for t in okt.pos(doc, norm=True, stem=True)]
            # norm은 정규화(오타 및 띄어쓰기 교정), stem은 근어화
        data['comment'] = data['comment'].apply(tokenize)
        print("// 형태소 전처리 진행")
        pos_tag = [(data['comment'][i], int(data['risk'][i])) for i in range(len(data))]
        # JSON 파일로 저장
        with open(fname, 'w', encoding="utf-8") as f:
            json.dump(pos_tag, f, ensure_ascii=False, indent="\t")
            print("// 형태소 전처리 파일 생성 완료")
    return data

In [81]:
def word_select(data, num=10000):
    fname = "word_index.json"
    tokens = [token for row in data['comment'] for token in row]
    print("// 고유 단어 :", len(set(tokens)))
    text = nltk.Text(tokens, name='NMSC')
    selected_words = [t[0] for t in text.vocab().most_common(num)]
    with open(fname, 'w', encoding="utf-8") as f:
        json.dump(selected_words, f, ensure_ascii=False, indent="\t")
        print("// 단어 인덱스 파일 생성 완료")
    return selected_words

In [80]:
def counter_vect(data, word_index):
    print("// 워드 인덱스 정리 & 데이터 float화 진행")
    def term_frequency(doc):
        return [doc.count(word) for word in word_index]
    pre_data = np.asarray([term_frequency(d) for d in list(data['comment'])]).astype('float32')
    pre_label = data['risk'].astype('float32')
    print("// 전처리 최종 완료")
    return (pre_data, pre_label)

In [92]:
def deep_learning(dataset):
    train_data, test_data, train_label, test_label = train_test_split(dataset[0], dataset[1])
    
    model = Sequential()
    model.add(Dense(64, activation='relu', input_shape=(train_data.shape[1],)))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer=optimizers.RMSprop(lr=0.001),
             loss=losses.binary_crossentropy,
             metrics=[metrics.binary_accuracy])

    model.fit(train_data, train_label, epochs=100,batch_size=512)   
    print("// 평가 :", model.evaluate(test_data, test_label))
    print("// 모델 저장 완료") 
    model.save('model.h5')
    return model

In [82]:
data = check_pos()
word_index = word_select(data)
dataset = counter_vect(data, word_index)

// 형태소 전처리 파일 존재
// 고유 단어 :  20752
// 단어 인덱스 파일 생성 완료
// 워드 인덱스 정리 & 데이터 float화 진행
// 전처리 최종 완료


In [93]:
model = deep_learning(dataset)

Epoch 1/100
35040/35040 [==============================] - 5s 148us/step - loss: 0.4469 - binary_accuracy: 0.8199
Epoch 2/100
35040/35040 [==============================] - 5s 142us/step - loss: 0.2958 - binary_accuracy: 0.8895
Epoch 3/100
35040/35040 [==============================] - 5s 146us/step - loss: 0.2518 - binary_accuracy: 0.9023
Epoch 4/100
35040/35040 [==============================] - 5s 144us/step - loss: 0.2252 - binary_accuracy: 0.9129
Epoch 5/100
35040/35040 [==============================] - 5s 141us/step - loss: 0.2034 - binary_accuracy: 0.9213
Epoch 6/100
35040/35040 [==============================] - 5s 143us/step - loss: 0.1845 - binary_accuracy: 0.9312
Epoch 7/100
35040/35040 [==============================] - 5s 144us/step - loss: 0.1670 - binary_accuracy: 0.9377
Epoch 8/100
35040/35040 [==============================] - 5s 142us/step - loss: 0.1504 - binary_accuracy: 0.9451
Epoch 9/100
35040/35040 [==============================] - 5s 136us/step - loss: 0.1355 

35040/35040 [==============================] - 5s 150us/step - loss: 0.0236 - binary_accuracy: 0.9911
Epoch 71/100
35040/35040 [==============================] - 5s 144us/step - loss: 0.0237 - binary_accuracy: 0.9910
Epoch 72/100
35040/35040 [==============================] - 5s 150us/step - loss: 0.0238 - binary_accuracy: 0.9915
Epoch 73/100
35040/35040 [==============================] - 5s 149us/step - loss: 0.0235 - binary_accuracy: 0.9912
Epoch 74/100
35040/35040 [==============================] - 5s 150us/step - loss: 0.0234 - binary_accuracy: 0.9911
Epoch 75/100
35040/35040 [==============================] - 5s 146us/step - loss: 0.0233 - binary_accuracy: 0.9912
Epoch 76/100
35040/35040 [==============================] - 5s 148us/step - loss: 0.0235 - binary_accuracy: 0.9913
Epoch 77/100
35040/35040 [==============================] - 5s 150us/step - loss: 0.0235 - binary_accuracy: 0.9912
Epoch 78/100
35040/35040 [==============================] - 5s 148us/step - loss: 0.0232 - bi

In [96]:
model2 = load_model("model.h5")

In [123]:
def predict_risk(doc, word_index):
    okt = Okt()
    def tokenize(doc):
        return ['/'.join(t) for t in okt.pos(doc, norm=True, stem=True)]
        # norm은 정규화(오타 및 띄어쓰기 교정), stem은 근어화
    token = tokenize(doc)
    data = np.expand_dims(np.asarray([token.count(word) for word in word_index]).astype('float32'), axis=0)
    score = float(model.predict(data))
    print("[{}]의 위험 지수는 {:.2f}% 입니다.\n".format(doc, score * 100))

In [124]:
predict_risk("오늘 정말 좋은 날이네요", word_index)

[오늘 정말 좋은 날이네요]의 위험 지수는 0.00% 입니다.

